In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import sqlite3 # library for working with sqlite database

### GETTING TABLES

In [2]:
# Create connections to the on-disk databases
conn2 = sqlite3.connect("./data/lab_events.db") # keep this
conn3 = sqlite3.connect("./data/labevents_common.db") # change this

# Create dataframes
#lab_events_df = pd.read_sql("""SELECT * FROM lab_events""", conn2)
lab_events_df = pd.read_sql("""SELECT * FROM labevents_common""", conn3)
lab_items_df = pd.read_sql("""SELECT * FROM lab_items""", conn2)

In [3]:
# Convert to numeric data types
lab_events_df["SUBJECT_ID"] = pd.to_numeric(lab_events_df["SUBJECT_ID"])
lab_events_df["HADM_ID"] = pd.to_numeric(lab_events_df["HADM_ID"]).astype(int)
lab_events_df["VALUENUM"] = pd.to_numeric(lab_events_df["VALUENUM"])

### CLEAN DATA

In [4]:
# Remove rows where VALUENUM is nan
lab_events_df = lab_events_df.loc[(lab_events_df["VALUENUM"]).notna(), :]

# Remove negative values
#lab_events_df = lab_events_df.loc[lab_events_df["VALUENUM"] >= 0,:]

# Merge with labels for laboratory measurements
lab_events_df = lab_events_df.merge(lab_items_df[['ITEMID','LABEL']], left_on='ITEMID', right_on='ITEMID')

### TRUTH LABELS

In [5]:
# Column 1: SUBJECT_ID
# Column 2: TRUTH
# Column 3: HADM_ID
truth_array = np.loadtxt("csv/TRUTH.csv", delimiter=',', skiprows=1)

# Create a dataframe with truth values
truth_df = pd.DataFrame(truth_array, columns=['SUBJECT_ID', 'TRUTH', 'HADM_ID'])

# Convert to ints
truth_df["SUBJECT_ID"] = truth_df["SUBJECT_ID"].astype(int)
truth_df["TRUTH"] = truth_df["TRUTH"].astype(int)
truth_df["HADM_ID"] = truth_df["HADM_ID"].astype(int)
#truth_df = truth_df.loc[truth_df["TRUTH"]==1]

### GET RELEVANT ADMISSIONS

In [6]:
# Only interested in lab events corresponding to the relevant admissions
relevant_lab_events_df = lab_events_df.merge(truth_df, on='HADM_ID')

### GET FEATURES

In [7]:
meas_type = np.unique(relevant_lab_events_df["LABEL"].get_values())
subjects = np.unique(relevant_lab_events_df["SUBJECT_ID_x"].get_values())

In [8]:
# Take the mean of every measurement for every subject
avg_df = relevant_lab_events_df.groupby(['HADM_ID', 'LABEL'], as_index = False)['VALUENUM'].median()
avg_df = avg_df.rename(index=str, columns={"VALUENUM": "AVG"})

In [9]:
# Create separate columns for glucose average, oxygen saturation average, and temperature average
design_mat_df = truth_df
for meas in meas_type:
    # Get avg of current measurement
    meas_avg_df = avg_df.loc[avg_df["LABEL"] == meas, ["HADM_ID", "AVG"]]
    
    # Rename
    rename_as = meas + "_AVG"
    meas_avg_df = meas_avg_df.rename(index=str, columns={"AVG": rename_as})
    
    # Merge
    design_mat_df = design_mat_df.merge(meas_avg_df, how='left', on='HADM_ID')


#### Missing Values

In [10]:
# Create additional columns for missing values
#for meas in meas_type:
#    # Get current column
#    curr_meas = design_mat_df.loc[:, meas + "_AVG"]
    
#    # Add column for missing values
#    design_mat_df[meas + "_AVG_missing"] = curr_meas.isna().astype(int)


In [11]:
# Median Imputation
design_mat_df = design_mat_df.fillna(design_mat_df.median())

In [12]:
#design_mat_df.iloc[:,3:len(design_mat_df.columns)].isna().sum().sort_values()

In [13]:
# Get X and y values
X = design_mat_df.iloc[:, 3:len(design_mat_df.columns)].get_values()
y = design_mat_df.TRUTH.get_values()

In [14]:
import random

# Get negative and positive examples
X0 = X[y == 0, :]
y0 = y[y == 0]
X1 = X[y == 1, :]
y1 = y[y == 1]

# Randomly select X0's
rand_idx = random.sample(range(0, len(y0)), len(y1))
X0_subset = X0[rand_idx, :]
y0_subset = y0[rand_idx]

# Recreate X and y
X = np.concatenate((X0_subset, X1))
y = np.concatenate((y0_subset, y1))

# Randomly order
rand_idx2 = random.sample(range(0, len(y)), len(y))
X = X[rand_idx2, :]
y = y[rand_idx2]

In [15]:
not_readmitted = design_mat_df.loc[design_mat_df["TRUTH"]==0, "HADM_ID"]
idx = random.sample(range(0, len(not_readmitted)), len(y1))
subset = not_readmitted.iloc[idx]

np.savetxt("Class0Subset.csv", subset, fmt='%d', delimiter="''")

### VISUALIZATION

In [16]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot
plt.scatter(X_pca[y==0, 0], X_pca[y==0, 1])
plt.scatter(X_pca[y==1, 0], X_pca[y==1, 1])
plt.show()

<Figure size 640x480 with 1 Axes>

### CLASSIFICATION

In [17]:
from sklearn.model_selection import train_test_split

# make train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.33, random_state = 0)

In [18]:
print(X_train)

[[40.5   10.4    4.76   5.4  ]
 [25.    10.4    3.03   4.8  ]
 [28.65   8.6    3.91  10.3  ]
 ...
 [35.6   12.7    3.83   8.8  ]
 [36.6   14.3    4.415 13.85 ]
 [25.2   10.4    2.57   8.3  ]]


In [19]:
from sklearn.preprocessing import normalize

# Normalize
#X_train[:, 0:3] = normalize(X_train[:, 0:3], norm='max', axis=0)
#X_test[:, 0:3] = normalize(X_test[:, 0:3], norm='max', axis=0)

In [20]:
from sklearn.svm import SVC

svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
y_pred = svm.decision_function(X_test)


In [21]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
AUC = metrics.auc(fpr, tpr)

In [22]:
#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(random_state=0)
#clf=clf.fit(X_train,y_train)
#y_pred = clf.predict(X_test)

In [23]:
AUC

0.5654528394810135